In [7]:
#Precision Recall f score 

###############################################
#Import Packages
###############################################
import pandas as pd
import numpy as np
from matplotlib import pyplot
import xlsxwriter
import os 
cwd = os.getcwd()
os.chdir(cwd)

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
pd.options.mode.chained_assignment = None

results = [
    r".\9_500_500.xlsx",
    r".\9_750_250.xlsx",
    r".\9_1000_0.xlsx",
    r".\6_500_500.xlsx",
    r".\6_750_250.xlsx",
    r".\6_1000_0.xlsx",
    r".\3_500_500.xlsx",
    r".\3_750_250.xlsx",
    r".\3_1000_0.xlsx"   
]

meta = [
    r".\data\9_500_500_meta.tsv",
    r".\data\9_750_250_meta.tsv",
    r".\data\9_1000_0_meta.tsv",
    r".\data\6_500_500_meta.tsv",
    r".\data\6_750_250_meta.tsv",
    r".\data\6_1000_0_meta.tsv",
    r".\data\3_500_500_meta.tsv",
    r".\data\3_750_250_meta.tsv",
    r".\data\3_1000_0_meta.tsv"   
]

###############################################
###P-R f measure and AUC function
###############################################
#input a=results b=meta
#output F1 score
def func(a, b):
    #Step 1: Input files and preprocess
    ###############################################    
    #Add new column and assign 1 or 0.
    results = pd.read_excel(a)
    meta = pd.read_csv(b, sep='\t')
    
    #for VoomLimma files
    results.rename(columns={'adj.P.Val': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    #for EdgeR files
    results.rename(columns={'FDR': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    
    #Assigning up/down regulation indicator
    ############################################### 
    #creates new column "sig"
    #uses pvalue and logfold change to assign 1 or 0
    #1000_1000 samples
    if a[-8:-5] == "0_0":
        results['sig']=''       
        results.loc[0:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #500_500 samples
    if a[-8:-5] == "500":
        results['sig']=''
        results.loc[0:499,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[500:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #750_250 samples
    if a[-8:-5] == "250":
        results['sig']=''
        results.loc[0:749,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[750:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0

    #print(results)
    ############################################### 
    
    
    #Step 2: Calculate f1 score
    ###############################################
    y_true = results[['sig']]
    y_pred = np.ravel(meta[['differential.expression']])
    
    f1 = f1_score(y_true, y_pred)
    
    return f1

###############################################
#generate list of results
###############################################
resultsList = []
for i in range(len(results)):
    
    result = func(results[i],meta[i])
    resultsList.append([results[i],result])

###############################################
###Output results
###############################################
#Input: resultslist
#output: Results.xlsx

workbook = xlsxwriter.Workbook('Results.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column(0, 8, 16)

for i in range(len(resultsList)):
    worksheet.write(i,0, resultsList[i][0])
    worksheet.write(i,1, resultsList[i][1])
workbook.close()

print(resultsList)

[['.\\9_500_500.xlsx', 0.6111111111111112], ['.\\9_750_250.xlsx', 0.7301587301587301], ['.\\9_1000_0.xlsx', 0.8452655889145496], ['.\\6_500_500.xlsx', 0.6008403361344538], ['.\\6_750_250.xlsx', 0.7192307692307692], ['.\\6_1000_0.xlsx', 0.858447488584475], ['.\\3_500_500.xlsx', 0.5621854780733285], ['.\\3_750_250.xlsx', 0.7236758136566688], ['.\\3_1000_0.xlsx', 0.8425925925925926]]


In [37]:
#Type I error rates in the absence of true differential expression. (FALSE POSITIVE)
#number of genes with p-value < 0.01
###################################################################################

#Confision matrix
from sklearn.metrics import confusion_matrix

#input a=results b=meta
#output F1 score
def func(a, b):
    #Step 1: Input files and preprocess
    ###############################################    
    #Add new column and assign 1 or 0.
    results = pd.read_excel(a)
    meta = pd.read_csv(b, sep='\t')
    
    #for VoomLimma files
    results.rename(columns={'P.Value': 'pvalue'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    #for EdgeR files
    results.rename(columns={'PValue': 'pvalue'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    
    #Assigning up/down regulation indicator
    ############################################### 
    #creates new column "sig"
    #uses pvalue and logfold change to assign 1 or 0
    #1000_1000 samples
    if a[-8:-5] == "0_0":
        results['sig']=''       
        results.loc[0:999,].loc[(results.pvalue <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #500_500 samples
    if a[-8:-5] == "500":
        results['sig']=''
        results.loc[0:499,].loc[(results.pvalue <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[500:999,].loc[(results.pvalue <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #750_250 samples
    if a[-8:-5] == "250":
        results['sig']=''
        results.loc[0:749,].loc[(results.pvalue <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[750:999,].loc[(results.pvalue <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0

    #print(results)
    ############################################### 
    
    
    #Step 2: Confusion matrix
    ###############################################
    
    y_true = results[['sig']]
    y_pred = np.ravel(meta[['differential.expression']])
    
    cm = confusion_matrix(y_true, y_pred)
    false_positive = cm[0][1]
    false_positive_proportion = false_positive/results.shape[0]
    print(false_positive_proportion)
    return false_positive_proportion

###############################################
#generate list of results
###############################################
resultsList = []
for i in range(len(results)):
    
    result = func(results[i],meta[i])
    resultsList.append([results[i],result])

###############################################
###Output results
###############################################
#Input: resultslist
#output: Results.xlsx

workbook = xlsxwriter.Workbook('Results.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column(0, 8, 16)

for i in range(len(resultsList)):
    worksheet.write(i,0, resultsList[i][0])
    worksheet.write(i,1, resultsList[i][1])
workbook.close()

print(resultsList)


0.024007202160648194
0.023902390239023904
0.026002600260026
0.03230969290787236
0.032409722916875064
0.03361344537815126
0.04996495444077301
0.049754730203223546
0.04966953735229321
[['.\\9_500_500.xlsx', 0.024007202160648194], ['.\\9_750_250.xlsx', 0.023902390239023904], ['.\\9_1000_0.xlsx', 0.026002600260026], ['.\\6_500_500.xlsx', 0.03230969290787236], ['.\\6_750_250.xlsx', 0.032409722916875064], ['.\\6_1000_0.xlsx', 0.03361344537815126], ['.\\3_500_500.xlsx', 0.04996495444077301], ['.\\3_750_250.xlsx', 0.049754730203223546], ['.\\3_1000_0.xlsx', 0.04966953735229321]]


In [38]:
#Power to detect true differential expression.
#Total number of genes that are detected as statistically significant (FDR < 0.1) 
###################################################################################

#input a=results b=meta
#output F1 score
def func(a, b):
    #Step 1: Input files and preprocess
    ###############################################    
    #Add new column and assign 1 or 0.
    results = pd.read_excel(a)
    meta = pd.read_csv(b, sep='\t')
    
    #for VoomLimma files
    results.rename(columns={'adj.P.Val': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    #for EdgeR files
    results.rename(columns={'FDR': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    
    #Assigning up/down regulation indicator
    ############################################### 
    #creates new column "sig"
    #uses pvalue and logfold change to assign 1 or 0
    #1000_1000 samples
    if a[-8:-5] == "0_0":
        results['sig']=''       
        results.loc[0:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #500_500 samples
    if a[-8:-5] == "500":
        results['sig']=''
        results.loc[0:499,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[500:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #750_250 samples
    if a[-8:-5] == "250":
        results['sig']=''
        results.loc[0:749,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[750:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0

    #print(results)
    ############################################### 
    
    
    #Step 2: sum up number of p value adjusted genes < 0.05
    ###############################################
    sigResults = results['sig']
    fdr = sigResults.sum(axis=0)
    
    print(fdr)

    return fdr
###############################################
#generate list of results
###############################################
resultsList = []
for i in range(len(results)):
    
    result = func(results[i],meta[i])
    resultsList.append([results[i],result])

###############################################
###Output results
###############################################
#Input: resultslist
#output: Results.xlsx

workbook = xlsxwriter.Workbook('FDR_Sum_Results.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column(0, 8, 16)

for i in range(len(resultsList)):
    worksheet.write(i,0, resultsList[i][0])
    worksheet.write(i,1, resultsList[i][1])
workbook.close()

NameError: name 'y_true' is not defined

In [42]:
#False discovery rates. 
#The number of false discoveries is plotted for each method versus the number of genes selected as differentially expressed.

#input a=results b=meta
#output F1 score
def func(a, b):
    #Step 1: Input files and preprocess
    ###############################################    
    #Add new column and assign 1 or 0.
    results = pd.read_excel(a)
    meta = pd.read_csv(b, sep='\t')
    
    #for VoomLimma files
    results.rename(columns={'adj.P.Val': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    #for EdgeR files
    results.rename(columns={'FDR': 'padj'}, inplace=True)
    results.rename(columns={'logFC': 'log2FoldChange'}, inplace=True)
    
    
    #Assigning up/down regulation indicator
    ############################################### 
    #creates new column "sig"
    #uses pvalue and logfold change to assign 1 or 0
    #1000_1000 samples
    if a[-8:-5] == "0_0":
        results['sig']=''       
        results.loc[0:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #500_500 samples
    if a[-8:-5] == "500":
        results['sig']=''
        results.loc[0:499,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[500:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0
        
    #750_250 samples
    if a[-8:-5] == "250":
        results['sig']=''
        results.loc[0:749,].loc[(results.padj <= 0.05) & (results.log2FoldChange >= 0),'sig']=1
        results.loc[750:999,].loc[(results.padj <= 0.05) & (results.log2FoldChange <= 0),'sig']=1
        results.loc[results.sig != 1,'sig']=0

    #print(results)
    ############################################### 
    
    
    #Step 2: FDR calculation
    ###############################################
    y_true = results[['sig']]
    y_pred = np.ravel(meta[['differential.expression']])
    
    cm = confusion_matrix(y_true, y_pred)
    true_pos = cm[0][0]
    false_pos = cm[0][1]
    fdr = false_pos / (true_pos + false_pos)
    
    print(fdr)

    return fdr

###############################################
#generate list of results
###############################################
resultsList = []
for i in range(len(results)):
    
    result = func(results[i],meta[i])
    resultsList.append([results[i],result])

###############################################
###Output results
###############################################
#Input: resultslist
#output: Results.xlsx

workbook = xlsxwriter.Workbook('FDR_Sum_Results.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column(0, 8, 16)

for i in range(len(resultsList)):
    worksheet.write(i,0, resultsList[i][0])
    worksheet.write(i,1, resultsList[i][1])
workbook.close()

0.04348288326600042
0.041946130096880654
0.047724867724867726
0.05493120470538809
0.05324074074074074
0.05503623568952841
0.07379161084200762
0.07434867675831532
0.07399010717230008
